In [1]:
import mne
from scipy import stats
from mne.stats import bonferroni_correction, fdr_correction
from mne import Epochs, pick_types, events_from_annotations
from mne.channels import make_standard_montage
import pickle
import wget
import pandas as pd
import zipfile
import os
import shutil
import numpy as np
import time
from numpy.fft import rfft
from scipy import signal
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from itertools import combinations
from sklearn.model_selection import ShuffleSplit, cross_val_score

In [2]:
import pystmm

In [3]:
from scipy.signal import butter, lfilter, filtfilt

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = filtfilt(b, a, data)
    return y

In [4]:
#Retrived from: http://www.jtrive.com/the-empirical-bootstrap-for-confidence-intervals-in-python.html
def bootstrap(data, n=1000, func=np.mean):
    """
    Generate `n` bootstrap samples, evaluating `func`
    at each resampling. `bootstrap` returns a function,
    which can be called to obtain confidence intervals
    of interest.
    """
    simulations = list()
    sample_size = len(data)
    xbar_init = np.mean(data)
    for c in range(n):
        itersample = np.random.choice(data, size=sample_size, replace=True)
        simulations.append(func(itersample))
    simulations.sort()
    def ci(p):
        """
        Return 2-sided symmetric confidence interval specified
        by p.
        """
        u_pval = (1+p)/2.
        l_pval = (1-u_pval)
        l_indx = int(np.floor(n*l_pval))
        u_indx = int(np.floor(n*u_pval))
        return(simulations[l_indx],simulations[u_indx])
    return(ci)

In [5]:
def CARFiltering(df, channels_interest):
    dictData = df.to_dict(orient='list')
    for i in range(len(df)):
        meanVal = float(np.mean([float(dictData[col][i]) for col in channels_interest]))
        for col in channels_interest:
            dictData[col][i] = dictData[col][i] - meanVal
    df = pd.DataFrame(dictData)
    return df.copy()

In [6]:
def getDFData(URL, User):
    #Data Description: https://lampx.tugraz.at/~bci/database/001-2019/dataset_description_v1-1.pdf
    #Offline: Type run and event codes
    dictTypeRun =  {'Run 1':'EyeMovements',
                  'Run 2':'Rest',
                  'Run 3':'AttemptedMovement',
                  'Run 4':'AttemptedMovement',
                  'Run 5':'AttemptedMovement',
                  'Run 6':'AttemptedMovement',
                  'Run 7':'AttemptedMovement',
                  'Run 8':'EyeMovements',
                  'Run 9':'Rest',
                  'Run 10':'AttemptedMovement',
                  'Run 11':'AttemptedMovement',
                  'Run 12':'AttemptedMovement',
                  'Run 13':'AttemptedMovement',
                  'Run 14':'EyeMovements',
                  'Run 15':'Rest'
                 }
    dictEvents = dict(TrialStart = 0x300,
                    Beep = 0x311,
                    FixationCross = 0x312,
                    SupinationClassCue = 0x308,
                    PronationClassCue = 0x309,
                    HandOpenClassCue = 0x30B,
                    PalmarGraspClassCue = 0x39D,
                    LateralGraspClassCue = 0x39E)
    dictEvents = dict(zip([str(val) for val in list(dictEvents.values())],list(dictEvents.keys())))
    dictColNames = dict(zip(list(dictEvents.keys()), list(range(len(list(dictEvents.keys()))))))
  
    #Unzip on User folder
    try:
        shutil.rmtree(User)
        os.mkdir(User)
    except:
        os.mkdir(User)  
  
  
    if not os.path.exists(User+'.zip'):
        # Download file
        print('Downloading: ',User,' from ',URL+User+'.zip')
        filename = wget.download(URL+User+'.zip')
    else:
        filename = User+'.zip'


  
    with zipfile.ZipFile(filename, 'r') as zip_ref:
        zip_ref.extractall(User)
    RunFiles = [os.path.join(User,f) for f in os.listdir(User) if os.path.isfile(os.path.join(User, f)) and f.lower().endswith('.gdf')]

    #Prepare DataFrame
    listDFRaw = []
    typeRun = []
    numRun = []
    samplingFrequencyList = []


  
    raw_all = mne.concatenate_raws([mne.io.read_raw_gdf(f, preload=True) for f in RunFiles])
    for run in dictTypeRun:
        rfile = None
        for runfile in RunFiles:
            if run in runfile:
                rfile = runfile
                break
        if rfile is None:
            continue
    
        raw = mne.io.read_raw_gdf(rfile)
    
        samplingFrequencyList.append(raw.info['sfreq'])
        ch_names = raw.info['ch_names']
        dfData = pd.DataFrame(data=raw.get_data().T,columns=ch_names)
        dfData = dfData.dropna(how='all')
        dfData.fillna(method='bfill',inplace=True)
        dfData = dfData.reset_index()
        dfData = dfData[ch_names]

        events, dictEventsRun = mne.events_from_annotations(raw)
        dictEventsRun = dict(zip([val for val in list(dictEventsRun.values())],list(dictEventsRun.keys())))
        sampleTime, eventNum = list(events[:,0]), list(events[:,2])
        listEventsPerColumn = [[0]*len(dfData)]*len(dictColNames)
        listEventsPerColumn = np.array(listEventsPerColumn)
        for s, e in zip(sampleTime, eventNum):
            if dictEventsRun[e] in dictColNames:
                listEventsPerColumn[dictColNames[dictEventsRun[e]],s] = 1
        dfEvents = pd.DataFrame(data=listEventsPerColumn.T,columns=[dictEvents[val] for val in list(dictColNames.keys())])
        dfRaw = pd.concat([dfEvents.copy(),dfData.copy()],axis=1,ignore_index=True)
        dfRaw.columns = [dictEvents[val] for val in list(dictColNames.keys())] + ch_names
    

        listDFRaw.append(dfRaw.copy())
        typeRun += [dictTypeRun[run]]*len(dfData)
        numRun += [run]*len(dfData)

    shutil.rmtree(User) 

    #Build DataFrame and return
    df = pd.concat(listDFRaw, ignore_index=True)
    df['TypeRun'] = typeRun
    df['Run'] = numRun
    df = df[['Run','TypeRun']+[dictEvents[val] for val in list(dictColNames.keys())]+ch_names]
    return df, list(set(samplingFrequencyList)), raw_all

## 1. Setting

In [7]:
dictEvents = dict(TrialStart = 0x300,
                  Beep = 0x311,
                  FixationCross = 0x312,
                  SupinationClassCue = 0x308,
                  PronationClassCue = 0x309,
                  HandOpenClassCue = 0x30B,
                  PalmarGraspClassCue = 0x39D,
                  LateralGraspClassCue = 0x39E)
dictEvents

{'Beep': 785,
 'FixationCross': 786,
 'HandOpenClassCue': 779,
 'LateralGraspClassCue': 926,
 'PalmarGraspClassCue': 925,
 'PronationClassCue': 777,
 'SupinationClassCue': 776,
 'TrialStart': 768}

In [8]:
typeRun = 'AttemptedMovement'
typeRest = 'Rest'
eeg_channels = ['AFz', 'F3', 'F1', 'Fz', 'F2', 'F4', 'FFC5h', 'FFC3h', 'FFC1h', 'FFC2h', 
                'FFC4h', 'FFC6h', 'FC5', 'FC3', 'FC1', 'FCz', 'FC2', 'FC4', 'FC6', 'FCC5h', 
                'FCC3h', 'FCC1h', 'FCC2h', 'FCC4h', 'FCC6h', 'C5', 'C3', 'C1', 'Cz', 'C2', 
                'C4', 'C6', 'CCP5h', 'CCP3h', 'CCP1h', 'CCP2h', 'CCP4h', 'CCP6h', 'CP5', 
                'CP3', 'CP1', 'CPz', 'CP2', 'CP4', 'CP6', 'CPP5h', 'CPP3h', 'CPP1h', 'CPP2h', 
                'CPP4h', 'CPP6h', 'P5', 'P3', 'P1', 'Pz', 'P2', 'P4', 'P6', 'PPO1h', 'PPO2h', 'POz']

movementTypes = ['SupinationClassCue','PronationClassCue','HandOpenClassCue', 'PalmarGraspClassCue','LateralGraspClassCue']
colors = ['red','green', 'blue', 'indigo','yellow','gray']
lowcut = 0.3
highcut = 3
butter_order = 4
tmin = -2.0
tmax = 3.0
bootstrapiterations = 100
confidenceinterval = .95
channel = 'C2'
Users = ['P01','P02','P03','P04','P05','P06','P07','P08','P09','P10']

In [9]:
df, sfreq, raw_mne = getDFData(URL='http://bnci-horizon-2020.eu/database/data-sets/001-2019/', User=Users[0])
ch_types = {}
for ch in raw_mne.ch_names:
  if not ch in ['eog-l','eog-m','eog-r']:
    ch_types[ch] = 'eeg'
  else:
    ch_types[ch] = 'eog'
raw_mne.set_channel_types(ch_types)
montage = make_standard_montage('standard_1005')
raw_mne.set_montage(montage)

raw_mne, _ = mne.set_eeg_reference(raw_mne, 'average',copy=True)

Extracting EDF parameters from D:\DESARROLLO_PROYECTO_DOCTORADO\pySTMM\examples\P01\P01 Run 1.gdf...
GDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 22015  =      0.000 ...    85.996 secs...
Extracting EDF parameters from D:\DESARROLLO_PROYECTO_DOCTORADO\pySTMM\examples\P01\P01 Run 10.gdf...
GDF file detected


c:\python36\lib\site-packages\mne\io\edf\edf.py:1047: DeprecationWarning:

The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead



Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 77823  =      0.000 ...   303.996 secs...
Extracting EDF parameters from D:\DESARROLLO_PROYECTO_DOCTORADO\pySTMM\examples\P01\P01 Run 11.gdf...
GDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 77055  =      0.000 ...   300.996 secs...


c:\python36\lib\site-packages\mne\io\edf\edf.py:1047: DeprecationWarning:

The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead

c:\python36\lib\site-packages\mne\io\edf\edf.py:1047: DeprecationWarning:

The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead



Extracting EDF parameters from D:\DESARROLLO_PROYECTO_DOCTORADO\pySTMM\examples\P01\P01 Run 12.gdf...
GDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 77823  =      0.000 ...   303.996 secs...
Extracting EDF parameters from D:\DESARROLLO_PROYECTO_DOCTORADO\pySTMM\examples\P01\P01 Run 13.gdf...
GDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 77311  =      0.000 ...   301.996 secs...


...

Used Annotations descriptions: ['33536', '33544', '33545', '33547', '33554', '33693', '33694', '768', '776', '777', '779', '785', '786', '925', '926']
Extracting EDF parameters from D:\DESARROLLO_PROYECTO_DOCTORADO\pySTMM\examples\P01\P01 Run 13.gdf...
GDF file detected
Setting channel info structure...
Creating raw.info structure...


c:\python36\lib\site-packages\mne\io\edf\edf.py:1047: DeprecationWarning:

The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead



Used Annotations descriptions: ['33536', '33544', '33545', '33547', '33554', '33693', '33694', '768', '776', '777', '779', '785', '786', '925', '926']
Extracting EDF parameters from D:\DESARROLLO_PROYECTO_DOCTORADO\pySTMM\examples\P01\P01 Run 14.gdf...
GDF file detected
Setting channel info structure...
Creating raw.info structure...
Used Annotations descriptions: ['33536', '768', '785']
Extracting EDF parameters from D:\DESARROLLO_PROYECTO_DOCTORADO\pySTMM\examples\P01\P01 Run 15.gdf...
GDF file detected
Setting channel info structure...
Creating raw.info structure...
Used Annotations descriptions: ['33536', '33554', '768', '785', '786']


c:\python36\lib\site-packages\mne\io\edf\edf.py:1047: DeprecationWarning:

The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead

c:\python36\lib\site-packages\mne\io\edf\edf.py:1047: DeprecationWarning:

The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead



EEG channel type selected for re-referencing
Applying average reference.
Applying a custom EEG reference.


In [10]:
# Transforming events
picks = pick_types(raw_mne.info, meg=False, eeg=True, stim=False, eog=False, exclude='bads')
events_before, dictEventsRun = mne.events_from_annotations(raw_mne)
dictEventsRun = dict(zip( list(dictEventsRun.values()) , [int(val) for val in list(dictEventsRun.keys())] ))
events = events_before.copy()
for i in range(events_before.shape[0]):
  events[i,2]=dictEventsRun[events[i,2]]

Used Annotations descriptions: ['33536', '33544', '33545', '33547', '33554', '33693', '33694', '768', '776', '777', '779', '785', '786', '925', '926']


In [11]:
event_id = dictEvents
tmin=-2.0
tmax = 3.0
epochs = Epochs(raw_mne, events, event_id, tmin, tmax, proj=True, picks=picks,
                baseline=None, preload=True, event_repeated='merge')

Not setting metadata
Multiple event values for single event times found. Creating new event value to reflect simultaneous events.
Not setting metadata
729 matching events found
No baseline correction applied
0 projection items activated
Loading data for 729 events and 1281 original time points ...
0 bad epochs dropped


In [12]:
epochs

Number of events,729
Events,Beep: 3Beep/FixationCross/TrialStart: 363Beep/TrialStart: 3HandOpenClassCue: 72LateralGraspClassCue: 72PalmarGraspClassCue: 72PronationClassCue: 72SupinationClassCue: 72
Time range,-2.000 – 3.000 sec
Baseline,off


In [13]:
sfreq = sfreq[0]

In [14]:
df

,Run,TypeRun,TrialStart,Beep,FixationCross,SupinationClassCue,PronationClassCue,HandOpenClassCue,PalmarGraspClassCue,LateralGraspClassCue,...,Pz,P2,P4,P6,PPO1h,PPO2h,POz,eog-l,eog-m,eog-r
0,Run 1,EyeMovements,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Run 1,EyeMovements,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Run 1,EyeMovements,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Run 1,EyeMovements,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Run 1,EyeMovements,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
825457,Run 15,Rest,0,0,0,0,0,0,0,0,...,246.204025,-85.778038,52.868378,227.765839,16.364775,120.490631,-10.344620,206.598831,133.239761,121.533577
825458,Run 15,Rest,0,0,0,0,0,0,0,0,...,247.676437,-83.594765,55.566181,230.387131,13.589742,122.498627,-12.317351,205.477188,127.036034,130.723816
825459,Run 15,Rest,0,0,0,0,0,0,0,0,...,254.758224,-78.117180,60.776100,234.809143,24.378588,128.832916,-3.641040,211.308929,136.794952,144.096100
825460,Run 15,Rest,0,0,0,0,0,0,0,0,...,246.911957,-85.132523,53.239170,225.796234,15.378569,120.152214,-11.697984,207.435608,132.239380,130.547943


### 0. Which is a filtered version of Data?

In [15]:
maincols = ['Run','TypeRun','TrialStart','Beep','FixationCross']

eeg_channels_of_interest = ['AFz', 'F3', 'F1', 'Fz', 'F2', 'F4', 'FFC5h', 'FFC3h', 'FFC1h', 'FFC2h', 
                'FFC4h', 'FFC6h', 'FC5', 'FC3', 'FC1', 'FCz', 'FC2', 'FC4', 'FC6', 'FCC5h', 
                'FCC3h', 'FCC1h', 'FCC2h', 'FCC4h', 'FCC6h', 'C5', 'C3', 'C1', 'Cz', 'C2', 
                'C4', 'C6', 'CCP5h', 'CCP3h', 'CCP1h', 'CCP2h', 'CCP4h', 'CCP6h', 'CP5', 
                'CP3', 'CP1', 'CPz', 'CP2', 'CP4', 'CP6', 'CPP5h', 'CPP3h', 'CPP1h', 'CPP2h', 
                'CPP4h', 'CPP6h', 'P5', 'P3', 'P1', 'Pz', 'P2', 'P4', 'P6', 'PPO1h', 'PPO2h', 'POz']

movementTypes_of_interest = ['SupinationClassCue','PronationClassCue','HandOpenClassCue', 'PalmarGraspClassCue','LateralGraspClassCue']

In [16]:
dfFiltered = df[maincols+movementTypes_of_interest+eeg_channels_of_interest]
dfFiltered = CARFiltering(dfFiltered, eeg_channels_of_interest)
dfFiltered.head(5)

,Run,TypeRun,TrialStart,Beep,FixationCross,SupinationClassCue,PronationClassCue,HandOpenClassCue,PalmarGraspClassCue,LateralGraspClassCue,...,P5,P3,P1,Pz,P2,P4,P6,PPO1h,PPO2h,POz
0,Run 1,EyeMovements,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Run 1,EyeMovements,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Run 1,EyeMovements,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Run 1,EyeMovements,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Run 1,EyeMovements,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### **1. ¿How many "Trial Start" events do Runs of type Rest are in the data and what is the temporal distance between cosecutive "Trial Start" events?**

In [17]:
### Analysis
# Filter by TypeRun equal to Rest
dfRest = dfFiltered[dfFiltered['TypeRun']=='Rest']
dfRest.reset_index(inplace=True)
#Signal rest
signalRest = dfRest['TrialStart'].values.tolist()
#Retrieve indexes where signal equals 1
indexesOnes = [i for i,val in enumerate(signalRest) if val == 1]
timesOnes = [val/(sfreq) for val in indexesOnes]
consecutiveOnes = [timesOnes[i+1]-timesOnes[i] for i,val in enumerate(timesOnes) if i+1 < len(timesOnes)]

### Report
print('Report on REST:')
print('Total samples = %d'%(len(signalRest)))
print('Sampling frequency (Hz) = %d'%(sfreq))
print('Signal duration (s) = %.2f'%(len(signalRest)/sfreq))
print('Signal duration (min) = %.2f'%(len(signalRest)/(60.0*sfreq)))
print('Total trial start events = '+str(len(indexesOnes)))
print('Indexes (sample) = '+str(indexesOnes))
print('Times (s) = '+str(timesOnes))
print('Duration between consecutive ones (s) = '+str(consecutiveOnes))

Report on REST:
Total samples = 69129
Sampling frequency (Hz) = 256
Signal duration (s) = 270.04
Signal duration (min) = 4.50
Total trial start events = 3
Indexes (sample) = [2560, 25603, 48646]
Times (s) = [10.0, 100.01171875, 190.0234375]
Duration between consecutive ones (s) = [90.01171875, 90.01171875]


#### **2. What are 72 epochs associated with REST?**

R:/ From every "Trial Start" event we can slide window of duration: tmax - tmin.

In [18]:
#Estimating start and end for every epoch
totalEpochs = 72
epochsPerOnes = round(totalEpochs/len(indexesOnes))
starts = []
ends = []
startstimes = []
endstimes = []
offset = 0.5*sfreq # One half second
totalSamples = round(1 + ((tmax - tmin)*sfreq))
steps = round(0.5*totalSamples) #10% of advancing (90% traslaping)

for starIndexOnes in indexesOnes:
  for i in range(epochsPerOnes):
    start = round(starIndexOnes+offset+(i*steps))
    starts.append(start)
    ends.append(start+totalSamples)

#Transforming from sample to time
startstimes = [val/(sfreq) for val in starts]
endstimes = [val/(sfreq) for val in ends]

#Print and check
print(startstimes)
print(endstimes)

[10.5, 13.0, 15.5, 18.0, 20.5, 23.0, 25.5, 28.0, 30.5, 33.0, 35.5, 38.0, 40.5, 43.0, 45.5, 48.0, 50.5, 53.0, 55.5, 58.0, 60.5, 63.0, 65.5, 68.0, 100.51171875, 103.01171875, 105.51171875, 108.01171875, 110.51171875, 113.01171875, 115.51171875, 118.01171875, 120.51171875, 123.01171875, 125.51171875, 128.01171875, 130.51171875, 133.01171875, 135.51171875, 138.01171875, 140.51171875, 143.01171875, 145.51171875, 148.01171875, 150.51171875, 153.01171875, 155.51171875, 158.01171875, 190.5234375, 193.0234375, 195.5234375, 198.0234375, 200.5234375, 203.0234375, 205.5234375, 208.0234375, 210.5234375, 213.0234375, 215.5234375, 218.0234375, 220.5234375, 223.0234375, 225.5234375, 228.0234375, 230.5234375, 233.0234375, 235.5234375, 238.0234375, 240.5234375, 243.0234375, 245.5234375, 248.0234375]
[15.50390625, 18.00390625, 20.50390625, 23.00390625, 25.50390625, 28.00390625, 30.50390625, 33.00390625, 35.50390625, 38.00390625, 40.50390625, 43.00390625, 45.50390625, 48.00390625, 50.50390625, 53.00390625

In [19]:
epochsRest = []
labelsRest = []
for start,end in zip(starts,ends):
  epoch = dfRest.loc[start:end,eeg_channels_of_interest].values.T
  epochsRest.append(epoch.copy())
  labelsRest.append(0)
epochsRest = np.stack(epochsRest,axis=0)
labelsRest = np.array(labelsRest)

epochsRest.shape, labelsRest.shape

((72, 61, 1282), (72,))

#### **3. ¿How many epochs for -Lateral Grasp- are in the data and which are those epochs?**

In [20]:
### Analysis
# Filter by TypeRun equal to Rest
dfLG = dfFiltered[dfFiltered['TypeRun']=='AttemptedMovement']
dfLG.reset_index(inplace=True)
#Signal rest
signalLG = dfLG['LateralGraspClassCue'].values.tolist()
#Retrieve indexes where signal equals 1
indexesOnes = [i for i,val in enumerate(signalLG) if val == 1]
timesOnes = [val/(sfreq) for val in indexesOnes]
consecutiveOnes = [timesOnes[i+1]-timesOnes[i] for i,val in enumerate(timesOnes) if i+1 < len(timesOnes)]

### Report
print('Report on Lateral Grasp:')
print('Total samples = %d'%(len(signalLG)))
print('Sampling frequency (Hz) = %d'%(sfreq))
print('Signal duration (s) = %.2f'%(len(signalLG)/sfreq))
print('Signal duration (min) = %.2f'%(len(signalLG)/(60.0*sfreq)))
print('Total lateral grasp events = '+str(len(indexesOnes)))
print('Indexes (sample) = '+str(indexesOnes))
print('Times (s) = '+str(timesOnes))
print('Duration between consecutive ones (s) = '+str(consecutiveOnes))

Report on Lateral Grasp:
Total samples = 691044
Sampling frequency (Hz) = 256
Signal duration (s) = 2699.39
Signal duration (min) = 44.99
Total lateral grasp events = 72
Indexes (sample) = [8943, 17585, 24019, 29582, 32903, 46994, 61145, 64521, 80600, 82341, 87409, 93206, 110498, 126312, 128047, 142325, 166138, 167711, 171424, 183488, 186837, 217073, 219069, 222632, 231771, 244712, 248117, 258926, 264198, 278187, 283528, 287238, 316986, 320389, 335009, 338405, 345414, 347188, 349056, 356141, 401282, 408593, 413920, 415691, 428386, 433336, 450390, 453549, 467353, 475933, 477591, 501281, 503030, 511851, 523015, 530436, 539303, 555876, 566717, 574106, 577448, 581319, 589753, 602789, 620220, 623896, 631110, 635028, 640479, 652734, 680444, 682293]
Times (s) = [34.93359375, 68.69140625, 93.82421875, 115.5546875, 128.52734375, 183.5703125, 238.84765625, 252.03515625, 314.84375, 321.64453125, 341.44140625, 364.0859375, 431.6328125, 493.40625, 500.18359375, 555.95703125, 648.9765625, 655.121093

In [21]:
#Estimating start and end for every epoch
totalSamples = round(1 + ((tmax - tmin)*sfreq))
starts = []
ends = []
startstimes = []
endstimes = []

for starIndexOnes in indexesOnes:
  start = starIndexOnes + sfreq*tmin
  starts.append(start)
  ends.append(start+totalSamples)

#Transforming from sample to time
startstimes = [val/(sfreq) for val in starts]
endstimes = [val/(sfreq) for val in ends]

#Print and check
print(startstimes)
print(endstimes)

[32.93359375, 66.69140625, 91.82421875, 113.5546875, 126.52734375, 181.5703125, 236.84765625, 250.03515625, 312.84375, 319.64453125, 339.44140625, 362.0859375, 429.6328125, 491.40625, 498.18359375, 553.95703125, 646.9765625, 653.12109375, 667.625, 714.75, 727.83203125, 845.94140625, 853.73828125, 867.65625, 903.35546875, 953.90625, 967.20703125, 1009.4296875, 1030.0234375, 1084.66796875, 1105.53125, 1120.0234375, 1236.2265625, 1249.51953125, 1306.62890625, 1319.89453125, 1347.2734375, 1354.203125, 1361.5, 1389.17578125, 1565.5078125, 1594.06640625, 1614.875, 1621.79296875, 1671.3828125, 1690.71875, 1757.3359375, 1769.67578125, 1823.59765625, 1857.11328125, 1863.58984375, 1956.12890625, 1962.9609375, 1997.41796875, 2041.02734375, 2070.015625, 2104.65234375, 2169.390625, 2211.73828125, 2240.6015625, 2253.65625, 2268.77734375, 2301.72265625, 2352.64453125, 2420.734375, 2435.09375, 2463.2734375, 2478.578125, 2499.87109375, 2547.7421875, 2655.984375, 2663.20703125]
[37.9375, 71.6953125, 96.

In [22]:
epochsLG = []
labelsLG = []
for start,end in zip(starts,ends):
  epoch = dfLG.loc[start:end,eeg_channels_of_interest].values.T
  epochsLG.append(epoch.copy())
  labelsLG.append(1)
epochsLG = np.stack(epochsLG,axis=0)
labelsLG = np.array(labelsLG)

epochsLG.shape, labelsLG.shape

((72, 61, 1282), (72,))

#### **4. ¿How many epochs for -Palmar Grasp- are in the data and which are those epochs?**

In [23]:
### Analysis
# Filter by TypeRun equal to Rest
dfPG = dfFiltered[dfFiltered['TypeRun']=='AttemptedMovement']
dfPG.reset_index(inplace=True)
#Signal rest
signalPG = dfPG['PalmarGraspClassCue'].values.tolist()
#Retrieve indexes where signal equals 1
indexesOnes = [i for i,val in enumerate(signalPG) if val == 1]
timesOnes = [val/(sfreq) for val in indexesOnes]
consecutiveOnes = [timesOnes[i+1]-timesOnes[i] for i,val in enumerate(timesOnes) if i+1 < len(timesOnes)]

### Report
print('Report on Palmar Grasp:')
print('Total samples = %d'%(len(signalPG)))
print('Sampling frequency (Hz) = %d'%(sfreq))
print('Signal duration (s) = %.2f'%(len(signalPG)/sfreq))
print('Signal duration (min) = %.2f'%(len(signalPG)/(60.0*sfreq)))
print('Total palmar grasp events = '+str(len(indexesOnes)))
print('Indexes (sample) = '+str(indexesOnes))
print('Times (s) = '+str(timesOnes))
print('Duration between consecutive ones (s) = '+str(consecutiveOnes))

Report on Palmar Grasp:
Total samples = 691044
Sampling frequency (Hz) = 256
Signal duration (s) = 2699.39
Signal duration (min) = 44.99
Total palmar grasp events = 72
Indexes (sample) = [12348, 14235, 27539, 34798, 38215, 57730, 59516, 67881, 78581, 84314, 89274, 94755, 99695, 103158, 131762, 148100, 157071, 169570, 176737, 180043, 188644, 201256, 202986, 215257, 235335, 241319, 250064, 253413, 262245, 267448, 288830, 293662, 309221, 314984, 325424, 336807, 341870, 363078, 364909, 366953, 384610, 386392, 392179, 397461, 399408, 417314, 430162, 439103, 470766, 482849, 486695, 494129, 497341, 499340, 506444, 508236, 541263, 552146, 557493, 572224, 586069, 587947, 596857, 608283, 616990, 622069, 633053, 654408, 656011, 665559, 673363, 687156]
Times (s) = [48.234375, 55.60546875, 107.57421875, 135.9296875, 149.27734375, 225.5078125, 232.484375, 265.16015625, 306.95703125, 329.3515625, 348.7265625, 370.13671875, 389.43359375, 402.9609375, 514.6953125, 578.515625, 613.55859375, 662.3828125,

In [24]:
#Estimating start and end for every epoch
totalSamples = round(1 + ((tmax - tmin)*sfreq))
starts = []
ends = []
startstimes = []
endstimes = []

for starIndexOnes in indexesOnes:
  start = starIndexOnes + sfreq*tmin
  starts.append(start)
  ends.append(start+totalSamples)

#Transforming from sample to time
startstimes = [val/(sfreq) for val in starts]
endstimes = [val/(sfreq) for val in ends]

#Print and check
print(startstimes)
print(endstimes)

[46.234375, 53.60546875, 105.57421875, 133.9296875, 147.27734375, 223.5078125, 230.484375, 263.16015625, 304.95703125, 327.3515625, 346.7265625, 368.13671875, 387.43359375, 400.9609375, 512.6953125, 576.515625, 611.55859375, 660.3828125, 688.37890625, 701.29296875, 734.890625, 784.15625, 790.9140625, 838.84765625, 917.27734375, 940.65234375, 974.8125, 987.89453125, 1022.39453125, 1042.71875, 1126.2421875, 1145.1171875, 1205.89453125, 1228.40625, 1269.1875, 1313.65234375, 1333.4296875, 1416.2734375, 1423.42578125, 1431.41015625, 1500.3828125, 1507.34375, 1529.94921875, 1550.58203125, 1558.1875, 1628.1328125, 1678.3203125, 1713.24609375, 1836.9296875, 1884.12890625, 1899.15234375, 1928.19140625, 1940.73828125, 1948.546875, 1976.296875, 1983.296875, 2112.30859375, 2154.8203125, 2175.70703125, 2233.25, 2287.33203125, 2294.66796875, 2329.47265625, 2374.10546875, 2408.1171875, 2427.95703125, 2470.86328125, 2554.28125, 2560.54296875, 2597.83984375, 2628.32421875, 2682.203125]
[51.23828125, 58

In [25]:
epochsPG = []
labelsPG = []
for start,end in zip(starts,ends):
  epoch = dfPG.loc[start:end,eeg_channels_of_interest].values.T
  epochsPG.append(epoch.copy())
  labelsPG.append(2)
epochsPG = np.stack(epochsPG,axis=0)
labelsPG = np.array(labelsPG)

epochsPG.shape, labelsPG.shape

((72, 61, 1282), (72,))

#### 5. ¿How discriminable, on the basis of STM, are rest epochs from lateral grasp?

In [26]:
X = np.concatenate([epochsRest,epochsLG,epochsPG], axis=0)
y = np.concatenate([labelsRest,labelsLG,labelsPG])
X.shape, y.shape, y

((216, 61, 1282),
 (216,),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]))

In [27]:
clf_multi = pystmm.classifier.STMM(typemulticlassifier='ovr',C1=1.0, C2=1.0, maxIter=30, tolSTM=1e-4, penalty = 'l2', dual = True, tol=1e-4,loss = 'squared_hinge', maxIterSVM=100000)

maxIter  30
ovr 1.0 1.0 30 0.0001 l2 True 0.0001 squared_hinge 100000


In [28]:
cv = ShuffleSplit(10, test_size=0.2, random_state=42)
cv_split = cv.split(X)

In [29]:
scores = cross_val_score(clf_multi, X, y, cv=cv, n_jobs=1)
print(scores)

maxIter  30
ovr 1.0 1.0 30 0.0001 l2 True 0.0001 squared_hinge 100000
1 / 30
2 / 30
3 / 30
4 / 30


c:\python36\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning:

Liblinear failed to converge, increase the number of iterations.



5 / 30


c:\python36\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning:

Liblinear failed to converge, increase the number of iterations.



6 / 30


...

c:\python36\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning:

Liblinear failed to converge, increase the number of iterations.



29 / 30


c:\python36\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning:

Liblinear failed to converge, increase the number of iterations.



30 / 30
[0.47727273 0.5        0.61363636 0.52272727 0.56818182 0.54545455
 0.38636364 0.54545455 0.56818182 0.5       ]


c:\python36\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning:

Liblinear failed to converge, increase the number of iterations.

c:\python36\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning:

Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.



In [30]:
for i,j in cv_split:
  print(len(i),len(j))

172 44
172 44
172 44
172 44
172 44
172 44
172 44
172 44
172 44
172 44


In [31]:
from scipy.stats import binom

In [33]:
alpha = 0.05
c = 3
n = 44
x = (100/n)*binom.ppf(1-alpha, n, 1/c)
print(x)

45.45454545454546
